In [1]:
# Import the libraries
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Declare the credentials
client_id = '2a7adde1b07a4bf9a5fd750d6ff6f28d'
client_secret = '74a5c5859a0548ccae791419cc29c8fa'
redirect_uri='http://localhost:7777/callback'
username = 'ethan.falcao'

In [3]:
# Authorization flow
scope = 'user-top-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [4]:
#input artist uri 

artist_uri = '2YpbfG8YXAbcNHagReUjNX'

# Get all albums from the artist
albums = sp.artist_albums(artist_uri)

In [5]:
# List to store track IDs
track_ids = []
# Loop through albums and get track IDs
for album in albums['items']:
    album_id = album['id']
    
    # Get tracks from the album
    tracks = sp.album_tracks(album_id)
    
    # Extract and append track IDs
    for track in tracks['items']:
        track_id = track['id']
        track_ids.append(track_id)

In [6]:
track_ids

['1MDJamSqCj2Em1AFsTGWlO',
 '3GkKTAOxF80V91LkutANrl',
 '6VCONXk25lh3FOGmCvG3nz',
 '0y7GiY1q9FJqCWJw7seHtb',
 '2AIGAn1eTbonFQTWqgnQl0',
 '6JYPt5mGboBwBWjs2Fk97n',
 '0gNpwSDPCgsU1ihRmcpjp7',
 '3Mdqq45yVImKKQRz3EeiiI',
 '3fsPrb0e9gxkRDb56Kiv7V',
 '38HeVt28RzZPAZuoUjxj5Y',
 '61dJ4j2MM1644qvINUbLhF',
 '3L59iKLfkChzBV77PMo7DH']

In [7]:
len(track_ids)

12

In [8]:
def fetch_audio_features(sp, track_ids):
    # Make the API request
    audio_features = []
    index = 0
    while index < len(track_ids):
        audio_features += sp.audio_features(track_ids[index:index + 70])
        index += 70
    
    # Create an empty list to feed in different characteristics of the tracks
    features_list = []
        
    for track_id, features in zip(track_ids, audio_features):
        features_list.append([
        track_id,
        features['time_signature'], 
        features['danceability'],
        features['acousticness'],
        features['energy'],
        features['tempo'],
        features['instrumentalness'],
        features['loudness'],
        features['liveness'],
        features['duration_ms'],
        features['key'],
        features['valence'],
        features['speechiness'],
        features['mode']
    ])
        


    df_audio_features = pd.DataFrame(features_list, columns=['track_id','time_signature','danceability','acousticness', 'energy', 'tempo', 'instrumentalness', 'loudness', 
                                                             'liveness', 'duration_ms', 'key', 'valence',  'speechiness', 'mode'])

    
    return df_audio_features

In [9]:
ids_features =  fetch_audio_features(sp, track_ids)
ids_features.head()

,track_id,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness,mode
0,1MDJamSqCj2Em1AFsTGWlO,4,0.643,0.2930,0.495,120.985,0.000145,-12.750,0.214,164075,7,0.341,0.0928,0
1,3GkKTAOxF80V91LkutANrl,3,0.601,0.2950,0.648,129.925,0.000000,-12.326,0.273,115461,11,0.722,0.1020,0
2,6VCONXk25lh3FOGmCvG3nz,4,0.594,0.1560,0.604,127.015,0.000925,-13.198,0.115,124787,8,0.456,0.0493,0
3,0y7GiY1q9FJqCWJw7seHtb,5,0.551,0.3060,0.375,86.705,0.000209,-15.831,0.112,206838,5,0.348,0.1950,0
4,2AIGAn1eTbonFQTWqgnQl0,4,0.711,0.0558,0.508,136.982,0.000001,-12.369,0.110,140931,3,0.141,0.1160,0


In [10]:
song_info = []

for song_id in track_ids:
    try:
        # Search for the track using its ID
        track = sp.track(song_id)
        song_name = track['name']
        # Get the album name
        album_name = track['album']['name']
        # Get the artists a
        artists = [artist['name'] for artist in track['artists']]
        popularity = track['popularity']
        song_info.append({'ID': song_id, 
                          'Song Name': song_name, 
                          'Artists': artists, 
                          'Album Name': album_name, 
                          'Popularity': popularity})
    except:
        #time.sleep(.5)
        song_info.append({'ID': song_id, 'Song Name': 'Not Found'})
        track_ids.remove(song_id)
        pass

song_df = pd.DataFrame(song_info)
song_df    

,ID,Song Name,Artists,Album Name,Popularity
0,1MDJamSqCj2Em1AFsTGWlO,Crash!,"[Vowel Gang, Rimbo, ZeKe, SJ Barr]",CALCULATED,0
1,3GkKTAOxF80V91LkutANrl,!Yako,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,2
2,6VCONXk25lh3FOGmCvG3nz,timezone,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,3
3,0y7GiY1q9FJqCWJw7seHtb,casanueva,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,0
4,2AIGAn1eTbonFQTWqgnQl0,Rollin,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,1
5,6JYPt5mGboBwBWjs2Fk97n,WZRD,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,0
6,0gNpwSDPCgsU1ihRmcpjp7,(PURE),"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,1
7,3Mdqq45yVImKKQRz3EeiiI,LASTCART,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,1
8,3fsPrb0e9gxkRDb56Kiv7V,-BESTONE-,"[Vowel Gang, Rimbo, ZeKe, BIG JUICE]",CALCULATED,1
9,38HeVt28RzZPAZuoUjxj5Y,LONGTIME,"[Vowel Gang, SJ Barr, Zeke, Moss Beats]",LONGTIME,0


In [11]:
song_df = song_df.rename(columns={'ID': 'track_id',
                                  'Song Name': 'Song',
                                  'Album Name': 'Album'  })

In [12]:
df = pd.merge(song_df, ids_features, on='track_id')

In [13]:
df.head()

,track_id,Song,Artists,Album,Popularity,time_signature,danceability,acousticness,energy,tempo,instrumentalness,loudness,liveness,duration_ms,key,valence,speechiness,mode
0,1MDJamSqCj2Em1AFsTGWlO,Crash!,"[Vowel Gang, Rimbo, ZeKe, SJ Barr]",CALCULATED,0,4,0.643,0.2930,0.495,120.985,0.000145,-12.750,0.214,164075,7,0.341,0.0928,0
1,3GkKTAOxF80V91LkutANrl,!Yako,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,2,3,0.601,0.2950,0.648,129.925,0.000000,-12.326,0.273,115461,11,0.722,0.1020,0
2,6VCONXk25lh3FOGmCvG3nz,timezone,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,3,4,0.594,0.1560,0.604,127.015,0.000925,-13.198,0.115,124787,8,0.456,0.0493,0
3,0y7GiY1q9FJqCWJw7seHtb,casanueva,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,0,5,0.551,0.3060,0.375,86.705,0.000209,-15.831,0.112,206838,5,0.348,0.1950,0
4,2AIGAn1eTbonFQTWqgnQl0,Rollin,"[Vowel Gang, Rimbo, ZeKe]",CALCULATED,1,4,0.711,0.0558,0.508,136.982,0.000001,-12.369,0.110,140931,3,0.141,0.1160,0


In [63]:
# Replace '2YpbfG8YXAbcNHagReUjNX' with your actual Spotify artist ID
artist_id = '2YpbfG8YXAbcNHagReUjNX'

# Get the artist information from Spotify API
artist_info = sp.artist(artist_id)

# Extract genres from the artist information
genres = artist_info['genres']

print(f"Genres associated with the artist: {', '.join(genres)}")

Genres associated with the artist: 


In [48]:
genres

[]